In [ ]:
# upload the database.xlsx and flow_chart.png and then run all cells

In [1]:
!pip install gradio  # Install Gradio in Colab
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 

In [30]:
!ollama serve > server.log 2>&1 &

In [31]:
!ollama pull gemma2:2b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 7462734796d6... 100% ▕▏ 1.6 GB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling 097a36493f71... 100% ▕▏ 8.4 KB                         
pulling 2490e7468436... 100% ▕▏   65 B                         
pulling e18ad7af7efb... 100% ▕▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
!pip install pyautogen[ollama]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.5/455.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [24]:
import pandas as pd
database = pd.read_excel("database.xlsx")

database

,Unique code,link,Description
0,A,https://www.lycamobile.co.uk/en/bundles/sim-on...,This webpage offers a variety of SIM-only deal...
1,B,mylyca.lycamobile.co.uk/en/login/,Login webpage of lyca
2,C,https://www.lycamobile.co.uk/en/introducing-es...,Info about Esim
3,D,https://www.lycamobile.co.uk/en/quick-top-up/,The webpage provides an easy way to top up you...
4,E,https://www.lycamobile.co.uk,Website home page link
5,F,https://www.lycamobile.us/en/help/contact-us/,Link to customer service and contact info of c...


In [82]:
import gradio as gr
import os
import re
from autogen import AssistantAgent, UserProxyAgent
import pandas as pd

def train1(query):
    database = pd.read_excel("database.xlsx")
    gemma={
        "config_list" : [
        {
            "model": "gemma2:2b",
            "api_type": "ollama",
            "stream": False,
            "client_host": "http://127.0.0.1:11434",
        }
    ]
    }


    assistant = AssistantAgent("assistant", llm_config=gemma, system_message=f"""
    You are a tool designed to process user input and format it according to specific guidelines. Your task is to return the complete text provided by the user, while enclosing some words in square brackets and appending their corresponding unique code as a hyperlink. Follow these rules:

    Use the given dataframe to identify words or phrases and their unique codes:
    {database}

    Enclose in square brackets [ ] those words or phrases that relates with the description column in the dataframe, even if the words are not identical.

    Example: For input 'Visit the lyca studio website to download the app,' return:
    Visit the [lyca studio](D) website to download the app.
    prioritize the most relevant matches and keep the formatting natural.

    If no matches are found, return the text as is, without any brackets.

    Remember, your goal is to make the output clear and user-friendly while adhering to the dictionary mappings.""")

    user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

    # Start the chat
    mess=user_proxy.initiate_chat(
        assistant,
        message=f"format this: {query}",
        max_turns=1
    )


    def hyperlink(text, database):
        result = []
        for i in range(len(text)):
            if text[i] == "(" and i + 1 < len(text):
                result.append("(")
                # Replace the next character using the database
                link_value = database[database['Unique code'] == text[i+1]]['link'].iloc[0]
                result.append(link_value)  # Replace or keep original

            elif text[i] == ")":
                result.pop()
                result.append(")")
            else:
                result.append(text[i])
        return "".join(result)


    return hyperlink(mess.chat_history[1]["content"],database)

# Function to respond to the query
image_path = "/content/flow_chart.png"
def get_response(query, img):
    response = train1(query)
    return f"### Response:\n\n{response}"  # Format response as Markdown

In [84]:
def create_header():
    return """
    <h1 style="text-align: center; color: #4CAF50;">Hyperlink Agent</h1>
    <p style="text-align: center; font-size: 18px; color: #555;">
        Adds hyperlinks to the response
    </p>

    """

image_path = "/content/flow_chart.png"
# Gradio interface
iface = gr.Interface(
    fn=get_response,  # Function to handle the query
    inputs=[gr.Textbox(label="Input text:", placeholder="Type your text here..."),gr.Image(image_path, label="Flow Chart")],  # Input field
    outputs=gr.Markdown(label="Response:"),
    description=create_header()
)

# Launch the Gradio app inside the Colab notebook
iface.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1597a394ff7c9ebfe5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [64]:
train1("In essence, top up of credit is a way to keep your mobile phone working by maintaining a positive balance on your prepaid account. It's a convenient way to manage your phone expenses and stay connected without being tied to a monthly contract.   ")

user_proxy (to assistant):

format this: In essence, top up of credit is a way to keep your mobile phone working by maintaining a positive balance on your prepaid account. It's a convenient way to manage your phone expenses and stay connected without being tied to a monthly contract.   

--------------------------------------------------------------------------------
assistant (to user_proxy):

In essence, [top up of credit](D) is a way to keep your mobile phone working by maintaining a positive balance on your prepaid account. It's a convenient way to manage your phone expenses and stay connected without being tied to a monthly contract. 


--------------------------------------------------------------------------------


"In essence, [top up of credit](https://www.lycamobile.co.uk/en/quick-top-up/) is a way to keep your mobile phone working by maintaining a positive balance on your prepaid account. It's a convenient way to manage your phone expenses and stay connected without being tied to a monthly contract. \n"